# Task 1 : Static Thresholds

In [21]:
import pandas as pd
import numpy as np

class TradingStrategy:
    def __init__(self, build_threshold, liquidate_threshold):
        self.build_threshold = build_threshold
        self.liquidate_threshold = liquidate_threshold

    def generate_positions(self, data):
        positions = []
        current_position = 0

        for alpha in data['alpha']:
            if current_position == 0:
                if alpha >= self.build_threshold:
                    current_position = 1  # Buy
                elif alpha <= -self.build_threshold:
                    current_position = -1  # Sell
            elif current_position == 1:
                if alpha <= self.liquidate_threshold:
                    current_position = 0  # Liquidate
            elif current_position == -1:
                if alpha >= -self.liquidate_threshold:
                    current_position = 0  # Liquidate

            positions.append(current_position)

        data['position'] = positions
        return data

# Load the dataset
file_path = '/content/asset_1.csv'
data = pd.read_csv(file_path)

# Define fixed thresholds
build_threshold = 0.6
liquidate_threshold = 0.2

# Create and apply strategy
strategy = TradingStrategy(build_threshold, liquidate_threshold)
data_with_positions = strategy.generate_positions(data)

# Store the updated data
output_file_path = '/content/updated_asset_1.csv'
data_with_positions.to_csv(output_file_path, index=False)

# Display the first few rows of the updated dataset
print(data_with_positions.head())

   serial_num         price     alpha  position
0           0  18105.300781  0.000000         0
1           1  18398.960605 -0.630291        -1
2           2  18204.939538 -0.123420         0
3           3  18339.357782 -0.240239         0
4           4  18578.084798 -0.612625        -1


# Task 2: Backtesting Engine


In [20]:
class BacktestEngine:
    def __init__(self, data, strategy):
        self.data = data
        self.strategy = strategy

    def run_backtest(self):
        data_with_positions = self.strategy.generate_positions(self.data.copy())
        cash = 0
        position = 0
        entry_price = 0
        pnl = []

        for index, row in data_with_positions.iterrows():
            current_price = row['price']
            current_position = row['position']

            if position == 0:
                if current_position != 0:
                    position = current_position
                    entry_price = current_price
            else:
                if current_position == 0:
                    cash += position * (current_price - entry_price)
                    position = 0
                elif position != current_position:
                    cash += position * (current_price - entry_price)
                    position = current_position
                    entry_price = current_price

            pnl.append(cash)

        data_with_positions['pnl'] = pnl
        return cash, data_with_positions

# Load the dataset
file_path = '/content/asset_1.csv'
data = pd.read_csv(file_path)

# Define fixed thresholds
build_threshold = 0.6
liquidate_threshold = 0.2

# Create strategy and backtest engine
strategy = TradingStrategy(build_threshold, liquidate_threshold)
engine = BacktestEngine(data, strategy)

# Run backtest
final_pnl, data_with_pnl = engine.run_backtest()

# Save the updated data
output_file_path = '/content/pnl_asset_1.csv'
data_with_pnl.to_csv(output_file_path, index=False)

# Display the final P&L and the first few rows of the updated dataset
print("Final P&L:", final_pnl)
print(data_with_pnl.head())


Final P&L: 26250.881404344847
   serial_num         price     alpha  position         pnl
0           0  18105.300781  0.000000         0    0.000000
1           1  18398.960605 -0.630291        -1    0.000000
2           2  18204.939538 -0.123420         0  194.021067
3           3  18339.357782 -0.240239         0  194.021067
4           4  18578.084798 -0.612625        -1  194.021067


# Task 3 : Optimizing Thresholds

In [15]:
def optimize_thresholds(data, build_range, liquidate_range):
    best_pnl = -np.inf
    best_build_threshold = None
    best_liquidate_threshold = None

    for build_threshold in build_range:
        for liquidate_threshold in liquidate_range:
            strategy = TradingStrategy(build_threshold, liquidate_threshold)
            engine = BacktestEngine(data, strategy)
            pnl, _ = engine.run_backtest()

            if pnl > best_pnl:
                best_pnl = pnl
                best_build_threshold = build_threshold
                best_liquidate_threshold = liquidate_threshold

    return best_pnl, best_build_threshold, best_liquidate_threshold

# Define threshold ranges to test
build_threshold_range = np.arange(0.1, 1.0, 0.1)
liquidate_threshold_range = np.arange(0.1, 1.0, 0.1)

# Find optimal thresholds
best_pnl, best_build_threshold, best_liquidate_threshold = optimize_thresholds(data, build_threshold_range, liquidate_threshold_range)

# Display the results
print(f"Best P&L: {best_pnl}")
print(f"Best Build Threshold: {best_build_threshold}")
print(f"Best Liquidate Threshold: {best_liquidate_threshold}")


Best P&L: 30376.675024315406
Best Build Threshold: 0.2
Best Liquidate Threshold: 0.1
